In [3]:
import pandas as pd
pd.set_option("display.max_column",None)
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
df=pd.DataFrame(pd.read_csv("bankloan.csv"))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       700 non-null    int64  
 1   ed        700 non-null    int64  
 2   employ    700 non-null    int64  
 3   address   700 non-null    int64  
 4   income    700 non-null    int64  
 5   debtinc   700 non-null    float64
 6   creddebt  700 non-null    float64
 7   othdebt   700 non-null    float64
 8   default   700 non-null    int64  
dtypes: float64(3), int64(6)
memory usage: 49.3 KB


In [6]:
df.head(10)

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.36,5.01,1
1,27,1,10,6,31,17.3,1.36,4.00,0
2,40,1,15,14,55,5.5,0.86,2.17,0
3,41,1,15,14,120,2.9,2.66,0.82,0
4,24,2,2,0,28,17.3,1.79,3.06,1
5,41,2,5,5,25,10.2,0.39,2.16,0
6,39,1,20,9,67,30.6,3.83,16.67,0
7,43,1,12,11,38,3.6,0.13,1.24,0
8,24,1,3,4,19,24.4,1.36,3.28,1
9,36,1,0,13,25,19.7,2.78,2.15,0


In [7]:
xtrain = df.drop(["income"],axis=1)

In [8]:
xtrain.head(5)

,age,ed,employ,address,debtinc,creddebt,othdebt,default
0,41,3,17,12,9.3,11.36,5.01,1
1,27,1,10,6,17.3,1.36,4.00,0
2,40,1,15,14,5.5,0.86,2.17,0
3,41,1,15,14,2.9,2.66,0.82,0
4,24,2,2,0,17.3,1.79,3.06,1


In [9]:
ytrain = df.iloc[:,4]

In [10]:
ytrain.head(5)

0    176
1     31
2     55
3    120
4     28
Name: income, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
xtrain = ss.fit_transform(xtrain)

In [12]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

In [13]:
LR.fit(xtrain,ytrain) 

LinearRegression()

In [14]:
yred=LR.predict(xtrain)

In [15]:
from sklearn.metrics import r2_score,mean_squared_error
print(r2_score(ytrain,yred))
print(mean_squared_error(ytrain,yred))

0.7628042246476463
321.00917316992104


In [16]:
print(LR.coef_)
print(LR.intercept_)

[  2.64041911   5.39311442   8.27946577   0.12416787 -19.68832444
  13.09675187  20.36351264   0.99299483]
45.60142857142857


In [19]:
import dill

with open("incommodel.pki", "wb") as f:
    dill.dump(LR, f)

In [20]:
import flask
import pickle
import pandas as pd

In [22]:
with open(f'incommodel.pki','rb') as f:
    model = pickle.load(f)

In [ ]:
#Initialise the flask app
app= flask.Flask(_name_,template_folder= 'template')

In [ ]:
@app.route('/',methods = ['GET','POST'])
def main():
    if flask.request.method == 'GET':
        #just render the initial form, to get input.
        return(flask.render_template('main.html'))

    if flask.request.method == 'POST':
        age = flask.request.form['age']
        ed = flask.request.form['ed']
        employ = flask.request.form['employ']
        address = flask.request.form['address']
        debtinc = flask.request.form['debtinc']
        creddebt = flask.request.form['creddebt']
        othdebt = flask.request.form['othdebt']
        defaul = flask.request.form['default']

        #make Dataframe for model
        input_variables = pd.DataFrame([[age,ed,employ,address,debtinc,creddebt,othdebt,default]], 
                                        columns=['age', 'ed', 'employ', 'address', 'debtinc', 'creddebt', 'othdebt', 'default'],
                                        index=['input'])

        #Get Model's prediction
        prediction = model.predict(input_variables)[0]

        #Render the form againbut add in the prediction and remind user of the values they input before
        return flask.render_template('main.html',
                                             orignal_input = {'Age' :age,
                                                               'ed' :ed,
                                                               'employ' :employ,
                                                                'address' :address,
                                                                'debtinc' :debtinc,
                                                                'creddebt' :creddebt,
                                                                'othdebt' :othdebt,
                                                                'default' :default},
                                             result = prediction,)

        if _name_=='_main_':
             